# Trial run of custom neural network with a real life dataset

Dataset used: [Fraud detection](https://www.kaggle.com/volodymyrgavrysh/fraud-detection-bank-dataset-20k-records-binary)  bank dataset from kaggle

In [1]:
# imports

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from model import Sigmoid, Linear, SequentialNN
from utils import binary_cross_entropy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Getting data ready

In [2]:
df = pd.read_csv(r"./data/fraud_detection_bank_dataset.csv")
df.drop(['Unnamed: 0'], axis=1, inplace = True)

In [3]:
df.head() # 112 features, hence cant visualize with graphs

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
0,9,1354,0,18,0,1,7,9,0,0,...,0,0,0,1,1,0,0,0,49,1
1,0,239,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,55,1
2,0,260,0,4,0,3,6,0,0,0,...,0,0,0,1,1,0,0,0,56,1
3,17,682,0,1,0,0,8,17,0,0,...,0,1,0,1,1,0,0,0,65,1
4,1,540,0,2,0,1,7,1,0,0,...,0,0,0,1,1,0,0,0,175,1


In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
y = y.reshape((len(y), 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (16374, 112)
X_test shape: (4094, 112)
y_train shape: (16374, 1)
y_test shape: (4094, 1)


In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train = X_train.T
y_train = y_train.T 
X_test = X_test.T
y_test = y_test.T 

## Creating our model/network

Architechure used:

- Input layer (112. i.e, n_features nodes)
- Hidden layer 1: Linear + Sigmoid (5 nodes)
- Hidden layer 2: Linear + sigmoid (3 nodes)
- Output layer: Linear + sigmoid (1 node)

In [8]:
# define training constants
learning_rate = 0.1
number_of_epochs = 10000

# setting seed for reproducible results
np.random.seed(100) 

In [9]:
model = SequentialNN()

# layer 1
Z1 = Linear(input_shape=X_train.shape, n_out=5, method='xavier')
S1 = Sigmoid(Z1.z.shape)
model.add(Z1)
model.add(S1)

In [10]:
# layer 2
Z2= Linear(input_shape=S1.sig.shape, n_out= 3, method='norm_xavier')
S2= Sigmoid(Z2.z.shape)
model.add(Z2)
model.add(S2)

In [11]:
# output layer
Z3= Linear(input_shape=S2.sig.shape, n_out=1, method='norm_xavier')
S3= Sigmoid(Z3.z.shape)
model.add(Z3)
model.add(S3)

In [12]:
model.compile(cost_func=binary_cross_entropy, epochs=number_of_epochs, learning_rate=learning_rate)

In [13]:
model.fit(X_train, y_train)

epoch 0: cost = 1.1351353195619127
epoch 100: cost = 0.4626866301581009
epoch 200: cost = 0.4062156043617507
epoch 300: cost = 0.3676288114482234
epoch 400: cost = 0.3411057113157241
epoch 500: cost = 0.32263868819317376
epoch 600: cost = 0.30949348124673753
epoch 700: cost = 0.2998212414224082
epoch 800: cost = 0.29236419635135896
epoch 900: cost = 0.28631209203755964
epoch 1000: cost = 0.28121738149095116
epoch 1100: cost = 0.27682066509955905
epoch 1200: cost = 0.27293625998637344
epoch 1300: cost = 0.26936885392347815
epoch 1400: cost = 0.2661761513509777
epoch 1500: cost = 0.26326892428142207
epoch 1600: cost = 0.26061088310503294
epoch 1700: cost = 0.2581516396896799
epoch 1800: cost = 0.2558533072420329
epoch 1900: cost = 0.2537179371516241
epoch 2000: cost = 0.2517573025535986
epoch 2100: cost = 0.2499634390718284
epoch 2200: cost = 0.24831572417996464
epoch 2300: cost = 0.24678758498609418
epoch 2400: cost = 0.24533587853770134
epoch 2500: cost = 0.243905572838767
epoch 2600: 

In [14]:
p = model.predict(X_test)
p = p.flatten()
y_test = y_test.flatten()

In [15]:
# classification report
print(classification_report(y_test, p))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2999
           1       0.84      0.84      0.84      1095

    accuracy                           0.92      4094
   macro avg       0.89      0.89      0.89      4094
weighted avg       0.92      0.92      0.92      4094



In [16]:
# confusion matrix
print(confusion_matrix(y_test, p))

[[2829  170]
 [ 174  921]]


Therefore there are:
1. 2829 true positives,
2. 921 true negatives,
3. 170 false positives,
4. 174 false negatives,

predicted by our model

In [17]:
# accuracy score
print(f"accuracy of custom neural network is: {(accuracy_score(y_test, p)*100).round(2)}%")

accuracy of custom neural network is: 91.6%
